In [ ]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
#from types import NoneType
import requests
from json import dump

from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen

# constants
BASE_URL = "https://www.domain.com.au"


#number of pages for each suburb (maximum is 50, 999 properties)
N_PAGES = range(1, 50 + 1) # update this to your liking

#https://www.domain.com.au/rent/melbourne-region-vic/?sort=price-desc&page={1}


headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}



postcode = 3000
while postcode <= 3996:

    # begin code
    url_links = []
    property_metadata = defaultdict(dict)

    # generate list of urls to visit
    for page in N_PAGES:

        #the full url 
        url = BASE_URL + f"/rent/?postcode={postcode}&page={page}"

        #parse into a BeautifulSoup object
        bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.


        #Keith's Note: Some results for a postcode may not display up to N_PAGES pages.
        #in that case the findAll() applied to the results secion of bs_object would return
        #nothing (NoneType object)

        try: 
            index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )

        except AttributeError:
            #Keith's Notes: there's no more pages to search, might as well to just break this for loop. 

            #suggestion: if we break from this for loop, and page = 1

            break


        for link in index_links:
            # if its a property address, add it to the list
            if 'address' in link['class']:
                url_links.append(link['href'])

    print(f"url links obtained for postcode {postcode} page {page}")

    # for each url, scrape some basic metadata
    for property_url in url_links[1:]:

        bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text

        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]

        
        #Keith's note: DEBUGGING INDEX ERROR: try to ignore the IndexError, see if any other error pops up
        rooms_list = []
        for feature in bs_object.find("div", {"data-testid": "property-features"}).findAll("span", {"data-testid": "property-features-text-container"}):
            try:
                rooms_list.append(re.findall(r'\d\s[A-Za-z]+', feature.text)[0])
            except IndexError:
                pass

        property_metadata[property_url]['rooms'] = rooms_list
        
        #Keith's note: below is the code that was actually given but replaced by above
        # property_metadata[property_url]['rooms'] = [
        #     re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
        #         .find("div", {"data-testid": "property-features"}) \
        #         .findAll("span", {"data-testid": "property-features-text-container"})
        # ]

        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p>')

    print(f"postcode {postcode} done, saving in progress..")

    # output to example json in data/raw/
    with open(f'../data/raw/vic_properties_postcode_{postcode}.json', 'w') as f:
        dump(property_metadata, f)

    postcode += 1
    


